Import the libraries

In [24]:
import pandas as pd
from pycaret.classification import *

In [25]:
def classification_problem(s):
    if s >= 1:
        return 1
    else:
        return 0

In [26]:
df = pd.read_csv('../../data/Unified-class.csv', index_col=0)
print(df.shape)
#df_type = df['Type']

(47618, 71)


In [27]:
df['bug'] = df['bug'].apply(classification_problem)

In [28]:
df.head()

,Type,Name,LongName,Parent,Component,Path,Line,Column,EndLine,EndColumn,...,TNLM,TNLPA,TNLPM,TNLS,TNM,TNOS,TNPA,TNPM,TNS,bug
ID,,,,,,,,,,,,,,,,,,,,,
L70222,Class,BindingKey,org.eclipse.jdt.core.BindingKey,L92879,L103,org.eclipse.jdt.core/model/org/eclipse/jdt/cor...,25,1,267,2,...,13,0,13,0,13,49,0,13,0,0
L57358,Class,BufferChangedEvent,org.eclipse.jdt.core.BufferChangedEvent,L92879,L103,org.eclipse.jdt.core/model/org/eclipse/jdt/cor...,47,1,117,2,...,5,0,5,0,5,8,0,5,0,0
L117505,Class,CheckDebugAttributes,org.eclipse.jdt.core.CheckDebugAttributes,L92879,L103,org.eclipse.jdt.core/antadapter/org/eclipse/jd...,40,1,98,2,...,4,0,3,2,4,33,0,3,2,0
L66611,Class,ClasspathContainerInitializer,org.eclipse.jdt.core.ClasspathContainerInitial...,L92879,L103,org.eclipse.jdt.core/model/org/eclipse/jdt/cor...,44,1,366,2,...,15,2,15,0,15,21,5,15,0,0
L130930,Class,ClasspathVariableInitializer,org.eclipse.jdt.core.ClasspathVariableInitializer,L92879,L103,org.eclipse.jdt.core/model/org/eclipse/jdt/cor...,26,1,52,2,...,2,0,2,0,2,0,0,2,0,0


In [29]:
df = df.drop('LongName', 1)
df = df.drop('Name', 1)
df = df.drop('Parent', 1)
df = df.drop('Component', 1)
df = df.drop('Path', 1)
df = df.drop('Line', 1)
df = df.drop('EndLine', 1)
df = df.drop('Column', 1)
df = df.drop('EndColumn', 1)
df = df.drop('Type', 1)

In [30]:
df.head()

,CC,CCL,CCO,CI,CLC,CLLC,LDC,LLDC,LCOM5,NL,...,TNLM,TNLPA,TNLPM,TNLS,TNM,TNOS,TNPA,TNPM,TNS,bug
ID,,,,,,,,,,,,,,,,,,,,,
L70222,0.0,0,0,0,0.0,0.0,0,0,6,1,...,13,0,13,0,13,49,0,13,0,0
L57358,0.0,0,0,0,0.0,0.0,0,0,0,0,...,5,0,5,0,5,8,0,5,0,0
L117505,0.0,0,0,0,0.0,0.0,0,0,1,4,...,4,0,3,2,4,33,0,3,2,0
L66611,0.0,0,0,0,0.0,0.0,0,0,2,1,...,15,2,15,0,15,21,5,15,0,0
L130930,0.0,0,0,0,0.0,0.0,0,0,0,0,...,2,0,2,0,2,0,0,2,0,0


In [31]:
corr = df.corr()
threshold = 0.99


columns = np.full((corr.shape[0],), True, dtype=bool)
for i in range(corr.shape[0]):
    for j in range(i+1, corr.shape[0]):
        if corr.iloc[i,j] >= threshold:
            if columns[j]:
                columns[j] = False
selected_columns = df.columns[columns]
selected_columns
df = df[selected_columns]

In [32]:
#df['Type'] = df_type

In [33]:
df.shape

(47618, 55)

In [34]:
df.columns

Index(['CC', 'CCL', 'CCO', 'CI', 'CLC', 'LDC', 'LCOM5', 'NL', 'NLE', 'WMC',
       'CBO', 'CBOI', 'NII', 'NOI', 'RFC', 'AD', 'CD', 'CLOC', 'DLOC', 'PDA',
       'PUA', 'TCLOC', 'DIT', 'NOA', 'NOC', 'NOD', 'NOP', 'LLOC', 'LOC', 'NA',
       'NG', 'NLA', 'NLG', 'NLM', 'NLPA', 'NLPM', 'NLS', 'NM', 'NOS', 'NPA',
       'NPM', 'NS', 'TLLOC', 'TLOC', 'TNG', 'TNLA', 'TNLG', 'TNLM', 'TNLPM',
       'TNLS', 'TNM', 'TNOS', 'TNPM', 'TNS', 'bug'],
      dtype='object')

In [36]:
NUMERIC_FEATURES = list(df.columns[:-1])

Setup reported to Eduardo on the third meeting<br>
s = setup(data=df, target='bug', feature_selection=True, remove_multicollinearity=True, multicollinearity_threshold=0.85, train_size=0.75, feature_selection_method='boruta', fix_imbalance=True, fold=10)

In [37]:
s = setup(data=df, target='bug', feature_selection=True, remove_multicollinearity=True, multicollinearity_threshold=0.85, train_size=0.7, feature_selection_method='boruta', fix_imbalance=True, fold=10)

,Description,Value
0,session_id,196
1,Target,bug
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(47618, 55)"
5,Missing Values,False
6,Numeric Features,50
7,Categorical Features,4
8,Ordinal Features,False
9,High Cardinality Features,False


In [40]:
#best_model = compare_models(sort='f1', include=['rf', 'lightgbm', 'et', 'knn', 'gbc'])
top5_models = compare_models(n_select=5, sort='f1', include=['rf', 'lightgbm', 'et', 'knn', 'gbc'])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.8338,0.8184,0.4385,0.5601,0.4918,0.3942,0.3985,1.5900
et,Extra Trees Classifier,0.8329,0.7777,0.4341,0.5573,0.4878,0.3899,0.3944,1.1050
knn,K Neighbors Classifier,0.7106,0.7573,0.6890,0.3523,0.4662,0.2951,0.3264,0.8460
lightgbm,Light Gradient Boosting Machine,0.8408,0.8284,0.3765,0.6063,0.4643,0.3768,0.3917,0.4380
gbc,Gradient Boosting Classifier,0.8163,0.7987,0.4315,0.4995,0.4628,0.3528,0.3543,2.7630


In [41]:
tuned_top5 = [
  tune_model(
    i,
    n_iter=30,
    optimize="f1",
    search_library='optuna',
    choose_better=True,
    early_stopping=True)
  for i in top5_models
]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8099,0.8017,0.4824,0.4817,0.4821,0.3657,0.3657
1,0.8194,0.8231,0.5344,0.5072,0.5204,0.4093,0.4095
2,0.8211,0.8259,0.5389,0.5116,0.5249,0.4148,0.4150
3,0.8132,0.8261,0.5428,0.4910,0.5156,0.4003,0.4010
4,0.8253,0.8282,0.5145,0.5241,0.5193,0.4125,0.4125
5,0.8230,0.8378,0.5160,0.5176,0.5168,0.4085,0.4085
6,0.8121,0.8169,0.4992,0.4881,0.4936,0.3783,0.3783
7,0.8177,0.8192,0.5389,0.5028,0.5203,0.4079,0.4083
8,0.8087,0.8097,0.5084,0.4798,0.4937,0.3759,0.3762


In [42]:
blended_models = blend_models(
  tuned_top5,
  choose_better=True,
  optimize="f1")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7906,0.8115,0.6015,0.4472,0.5130,0.3833,0.3901
1,0.7993,0.8328,0.6473,0.4659,0.5419,0.4177,0.4269
2,0.7937,0.8314,0.6275,0.4546,0.5273,0.3996,0.4081
3,0.7917,0.8286,0.6590,0.4527,0.5367,0.4083,0.4202
4,0.8104,0.8347,0.6397,0.4872,0.5531,0.4356,0.4421
5,0.8034,0.8415,0.6290,0.4730,0.5400,0.4181,0.4250
6,0.7900,0.8269,0.6168,0.4474,0.5186,0.3886,0.3968
7,0.7964,0.8321,0.6336,0.4601,0.5331,0.4071,0.4156
8,0.7959,0.8171,0.6305,0.4589,0.5312,0.4048,0.4132


[LightGBM] [Warning] bagging_fraction is set=0.5904182101267933, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5904182101267933
[LightGBM] [Warning] feature_fraction is set=0.6636158687227987, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6636158687227987
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] bagging_fraction is set=0.5904182101267933, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5904182101267933
[LightGBM] [Warning] feature_fraction is set=0.6636158687227987, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6636158687227987
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] bagging_fraction is set=0.5904182101267933, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5904182101267933
[LightGBM] [Warning] feature_fracti

In [43]:
result = pull()

In [44]:
result.to_csv('../../results/bug/journal.csv')

In [45]:
pred = predict_model(blended_models)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.7923,0.8257,0.6316,0.4605,0.5326,0.4032,0.4116


In [46]:
final_best = finalize_model(blended_models)

[LightGBM] [Warning] bagging_fraction is set=0.5904182101267933, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5904182101267933
[LightGBM] [Warning] feature_fraction is set=0.6636158687227987, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6636158687227987
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] bagging_fraction is set=0.5904182101267933, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5904182101267933
[LightGBM] [Warning] feature_fraction is set=0.6636158687227987, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6636158687227987
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] bagging_fraction is set=0.5904182101267933, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5904182101267933
[LightGBM] [Warning] feature_fracti

In [47]:
final_best

VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight='balanced_subsample',
                                                     criterion='gini',
                                                     max_depth=10,
                                                     max_features=0.6791051160890675,
                                                     max_leaf_nodes=None,
                                                     max_samples=None,
                                                     min_impurity_decrease=1.2786345928428282e-09,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=3,
                                                     min_samples_split=9,
                                              